# CHAPTER8 Vuexでアプリケーションの状態を管理する

Vuexは、Vue.jsアプリケーションのための状態管理パターン＋ライブラリのこと。

特に大規模な開発になると、親子コンポーネント間のデータの受け渡しが、`props`と`$emit`のみで行うと、バケツリレーになってしまう。

Vuexはこの親子間通信をより容易にするためのライブラリ。

* 複数のコンポーネント間でデータを共有
* データの状態に関する処理を共通化
* 大きな状態管理をモジュール分割で最適化

## 8.42 シンプルなストア構造

Vuexは、アプリケーションの状態を管理するための`ストア`と呼ばれるものを作成する。
* `ストア`：アプリケーションの状態を保持する仮想的なデータベースのようなもの

`Vuex.Store`を使い、Vuexストアのインスタンスを作成する。

```js
// store.js
const store = new Vuex.Store({
    state: { //アプリケーションで使用するデータを定義する
        count: 0
    },
    mutations: {
        increment(state) {
            state.count++
        }
    }
});
export default store;
```
この`store`をimportすれば、定義したデータを読み込むことができる
```js
import store from '@/store.js' // @はsrcディレクトリを指すエイリアス
console.log(store.state.count) // 0
```

### storeをアプリケーションに組み込む

ルートコンポーネントのVueインスタンスに、`store`オプションを追加することで`$store`としてどこからでもアクセスできるようになる。

```js
// main.js
import store from '@/store.js'
new Vue({
    el: '#app',
    store, // storeを追加
    render: h => h(App)
});
```
全コンポーネントからは以下のようにアクセスできる。
```js
export default {
    methods: {
        increment() {
            this.$store.commit('increment') // storeのmutationsのincrementを呼び出す
            this.$store.state.count // storeのstateのcountを参照する
        }
    }
}
```

### Vuex内のインスタンス参照方法

Vue.jsのように`this`キーワードを使った参照はしない。実装に必要なプロパティ・メソッドは引数として渡される。
```js
// store.js
const store = new Vuex.Store({
    ...
    mutations: {
        increment(state) { // stateは引数として渡される
            state.count++
        },
        /**メソッドの定義にはアロー関数を使用可能 */
        increment2: state => state.count++,
    }
});
```

## 8.43 コアコンセプト

コンポーネントからアプリケーションの状態`State`を動かすには、Vuexを経由して、Vuex内の`Action`と`Mutation`を呼び出し、その中で`State`を変更する。

コンポーネントから直接`State`を変更することはできない。

### ステート`State`

Vueコンポーネントで言うところの`data`のようなもの。`State`は`mutations`を介してのみ変更できる。
```js
// store.js
const store = new Vuex.Store({
    state: {
        count: 0
    },
    mutations: {
        increment(state) {
            state.count++
        }
    }
});

//呼び出し側
store.state.count // 0
```

### ゲッター`getter`

`State`取得用の算出データ。Vueコンポーネントの`computed`と`methods`の中間のようなもの。

引数を渡せるが、`setter`の機能はない。
```js
// store.js
const store = new Vuex.Store({
    state: {
        count: 0
    },
    getters: {
        doubleCount(state) {
            return state.count * 2
        }
    }
});
```

### ミューテーション`mutation`

`State`を変更するためのメソッド。`State`の変更は`mutation`を介して行う。

Vueコンポーネントの`methods`と同じようなもの。

```js
// store.js
const store = new Vuex.Store({
    state: {
        count: 0
    },
    mutations: {
        increment(state, payload) { //stateとpayloadを引数として受け取る
            state.count++
        }
    }
});
```
### コミット`commit`

定義されている`mutation`を呼び出すためのメソッド。Vueコンポーネントの`$emit`と同じようなもの。

```js
store.commit('increment') // mutationのincrementを呼び出す

/**複数の引数がある場合、オブジェクトで1つのpayloadにまとめる */
store.commit('increment', { amount: 10 })
```

### アクション`action`
非同期処理を含めるためのメソッド。非同期通信を行い、その結果を`mutation`に渡す。

```js
// store.js
const store = new Vuex.Store({
    ...
    actions: {
        actionType({ commit }) { //引数にはcontextオブジェクトが渡される
            commit('increment') //mutationのincrementを呼び出す
        }
    }
});
```
第１引数の中身は以下の通り

```ts
interface Context {
    state: any; // store.stateと同じ,モジュール内ならローカルのstate
    rootState: any; // store.stateと同じ（ただしモジュール内に限る）
    commit: Function; // store.commitと同じ
    getters: any; // store.gettersと同じ,モジュール内ならローカルのgetters
    rootGetters: any; // store.gettersと同じ（ただしモジュール内に限る）
    dispatch: Function; // store.dispatchと同じ
}
```



## 8.44 コンポーネントでストアを使用しよう

srcの構造は以下とする
```bash
src
├── App.vue
├── components
│   ├── EditForm.vue
├── main.js
├── store.js
```

`store.js`の定義は以下とする
```js
// store.js
import Vue from 'vue'
import Vuex from 'vuex'
Vue.use(Vuex)

const store = new Vuex.Store({
    state: {
        message: 'Hello Vuex!'
    },
    /** messageを利用するgetter */
    getters: {
        message(state) {
            return state.message
        }
    },
    /** メッセージを使用するミューテーション */
    mutations: {
        setNessage(state, payload) {
            state.message = payload.message
        }
    },
    actions: {
        /** メッセージの更新 */
        doUpdate({ commit }, payload) {
            commit('setMessage', payload)
        }
    }
});
```

以上を`main.js`で読み込む
```js
// main.js
import Vue from 'vue'
import App from './App.vue'
import store from './store.js'

new Vue({
    el: '#app',
    store,
    render: h => h(App)
})
```

